In [57]:
%matplotlib inline
import pandas as pd
import numpy as np
import geopandas as gpd
import contextily as ctx
import matplotlib.pyplot as plt
import matplotlib as mpl 
#mpl.use("module://mplcairo.qt") 
from mplcairo import operator_t
from mpl_toolkits.axes_grid1 import make_axes_locatable
import subprocess
from lib.funcs import *

In [59]:
## Get the names and open data
renda_path = ('./data/CENSO/PessoaRenda_PB.csv') 

data = pd.read_csv(renda_path,                  
                sep=';', 
                encoding="utf-8",
                dtype={'Cod_setor': str,
                      'V020': float,
                      'V021': float,
                      'V022': float},
                   na_values = ['X'],
                lineterminator='\n')

In [60]:
## Open SHP
setores = gpd.read_file('./data/shp/jp_shp.shp')
setores = setores.to_crs(epsg=3857)

In [61]:
border = setores
border['new_column'] = 0
border = border.dissolve(by='new_column')
border = border[['geometry']]
border['geometry'] = border['geometry'].buffer(0.0001)

In [62]:
# Select only the data we are interested in
#v001 = Moradores em domicilios particular e domicilios coletivos
renda = data[['Cod_setor', 'V020', 'V021', 'V022']]
renda = renda.rename(columns={'V020': 'num_pessoas',
                             'V021': 'num_pessoas_rendimento',
                             'V022': 'rendimento_nominal'})

setores = setores[['geometry', 'NM_BAIRRO', 'CD_GEOCODI']]

In [63]:
populacao = pd.read_csv('./data/POPULACAO/populacao_parse.csv',
                       dtype={'Cod_setor': str})

In [64]:
#populacao.set_index('Cod_setor', inplace=True)
#renda.set_index('Cod_setor', inplace=True)


In [65]:
renda = pd.merge(populacao, renda, on='Cod_setor')

In [66]:
renda['num_pessoas'] = renda['MORADORES']

In [68]:
renda.drop(['MORADORES'], axis=1, inplace=True)

In [69]:
renda

,Cod_setor,num_pessoas,num_pessoas_rendimento,rendimento_nominal
0,250010605000001,1528,712.0,304733.0
1,250010605000002,500,273.0,177732.0
2,250010605000003,889,484.0,310937.0
3,250010605000004,860,372.0,141471.0
4,250010605000005,667,326.0,143194.0
...,...,...,...,...
5543,251720905000004,345,183.0,81061.0
5544,251720905000005,1105,525.0,212375.0
5545,251740705000001,876,469.0,219933.0
5546,251740705000002,603,368.0,135068.0


In [70]:
# Set the same inde
renda.set_index('Cod_setor', inplace=True)
setores.set_index('CD_GEOCODI', inplace=True)

In [71]:
setores = setores.join(renda)

In [72]:
bairros = setores.dissolve(by='NM_BAIRRO', aggfunc='sum')

In [73]:
bairros_interesse = ['Torre', 'Tambauzinho', 'Expedicionários', 'Miramar', 'Pedro Gondim', 'Estados']
setores_recorte = setores.loc[setores['NM_BAIRRO'].isin(bairros_interesse)]

In [74]:
setores_recorte['renda_per_capita'] = setores_recorte.apply(get_renda_per_capita, axis=1)
setores_recorte['renda_por_trabalhador'] = setores_recorte.apply(get_renda_por_trabalhador, axis=1)
bairros['renda_per_capita'] = bairros.apply(get_renda_per_capita, axis=1)
bairros['renda_por_trabalhador'] = bairros.apply(get_renda_por_trabalhador, axis=1)

<ipython-input-74-349ba082b907>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  setores_recorte['renda_per_capita'] = setores_recorte.apply(get_renda_per_capita, axis=1)
<ipython-input-74-349ba082b907>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  setores_recorte['renda_por_trabalhador'] = setores_recorte.apply(get_renda_por_trabalhador, axis=1)


In [75]:
setores_recorte.to_file('./data/RENDA/setores_recorte.shp')
bairros.to_file('./data/RENDA/bairros.shp')

In [76]:
export(setores_recorte, 'renda_setores_recorte', e)
export(bairros, 'renda_bairros')